# Generate Synthetic Data

In [ ]:
import argparse
import yaml
import pathlib
from typing import List, Optional
import yaml
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from scipy import linalg
from tqdm import tqdm

from dataloader import *
from models import *
from utils.builder import build_pipeline
from utils.builder import build_metrics
from utils.builder import build_models
from dataloader.RandomDataset import *
from frechet import DiscreteFrechet, euclidean
import pickle
if __name__ == "__main__":
  
    yaml_f = "/nas/k8s/dev/research/intern/jhmin/test_project/despeckle/CTD/configs/capella.yaml"
    weight_path =  "/nas/k8s/dev/research/intern/jhmin/test_project/despeckle/CTD/weights_sar/capella_230926-060602.pkl"
    with open(weight_path, 'rb') as f:
            weight = pickle.load(f)
    with open(yaml_f) as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    model = build_models(config['model']).cuda()
    target_dataset = SynDataset(config['dataset']['src_path'], weight = weight,
                                              pipelines = config['degradations'])
    # N = RandomDegradationDataset(config['dataset']['src_path'], config['degradations']).num_possible_bins



    


# Single Image Save

In [2]:
import os
import cv2

def mkdir(paths):
    if not os.path.exists(paths):
        os.makedirs(paths)
for x in range (len(target_dataset)):
    img = target_dataset[x]['input'].permute((1,2,0)).numpy()
    
    paths = target_dataset[x]['path'][0]
    img = np.clip(img,0,255).astype(np.uint8)
    paths = paths.split("/")
    paths[10] = "Synthesized_val"
    check = paths[:-1]
    check = "/".join(check)
    paths = "/".join(paths)

    mkdir(check)
    cv2.imwrite(paths,img)
    


# PLT로 만들기

In [8]:
import os
import cv2
import matplotlib.pyplot as plt
old_string = 'hr'
# dn_string = 'Capella_1m_test_DN'
lr_string = 'lr'
save_folder = './Synthesized/'

def mkdir(paths):
    if not os.path.exists(paths):
        os.makedirs(paths)
mkdir(save_folder)
data = {}
for x in range (len(target_dataset)):
    data[x] = {}
    for key in target_dataset[x]:
        if key != 'path':
            data[x][key] = np.clip((target_dataset[x][key].permute((1,2,0)).numpy()),0,255).astype(np.uint8)

    c_path = target_dataset[x]['path'][0].split("/")[:13]
    filename =target_dataset[x]['path'][0].split("/")[-1]
    # dn_path = [dn_string if item == old_string else item for item in c_path]
    # dn_path = "/".join(dn_path)
    # dn_path = os.path.join(dn_path)
    # real_lr_dn = cv2.imread(dn_path)
    lr_path = [lr_string if item == old_string else item for item in c_path]
    lr_path = "/".join(lr_path)
    lr_path = os.path.join(lr_path)
    real_lr = cv2.imread(lr_path)
    
    data[x]['real_lr'] = real_lr
    data[x]['path'] = filename
    # data[x]['real_dn'] = real_lr_dn



In [10]:
# 각 이미지의 'label', 'input', 'real_lr' 이미지를 표시하고 저장
for image_name, image_data in data.items():
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image_data['label'])
    axes[0].set_title('Label')
    axes[1].imshow(image_data['input'])
    axes[1].set_title('Input')
    axes[2].imshow(image_data['real_lr'])
    axes[2].set_title('real_lr')
    # axes[3].imshow(image_data['real_dn'])
    # axes[3].set_title('real_dn')
    
    for ax in axes:
        ax.axis('off')
    
    fig.suptitle(f'Label: {image_name}', fontsize=16)
    
    combined_image_filename = f'{image_data["path"].replace(".tif", ".png")}'
    combined_image_filename = os.path.join(save_folder, combined_image_filename)
    plt.savefig(combined_image_filename, bbox_inches='tight')
    plt.close()

print('Combined images saved successfully.')

Combined images saved successfully.


# Normalize TIF files.

In [ ]:
import glob
import os
import tifffile as tiff
dirname = "/nas/k8s/dev/research/intern/jhmin/test_project/Datasets/SAR/Capella_mm1_uint16/"
o_dir = "/nas/k8s/dev/research/intern/jhmin/test_project/Datasets/SAR/Capella_uint16_Norm/"


def make_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
def min_max_normalize(image, percentile=2):
    # Input: [H, W, C]
        # tif 값의 nodata != 0 인경우
    # _mask = np.mean.. line의 !=0 값을 바꿀것.
        # nodata = rasterio.open(filename).meta['nodata']
    if len(image.shape) != 3:
        image = np.expand_dims(image, -1) 
    image = image.astype('float32')

    percent_min = np.percentile(image, percentile, axis=(0, 1))
    percent_max = np.percentile(image, 100-percentile, axis=(0, 1))

    _mask = np.mean(image, axis=2) != 0
    if image.shape[1] * image.shape[0] - np.sum(_mask) > 0:
        mdata = np.ma.masked_equal(image, 0, copy=False)
        mdata = np.ma.filled(mdata, np.nan)
        percent_min = np.nanpercentile(mdata, percentile, axis=(0, 1))

    norm = (image-percent_min) / (percent_max - percent_min)
    norm[norm < 0] = 0
    norm[norm > 1] = 1
    norm = (norm * 255).astype('uint8') * _mask[:, :, np.newaxis]
    return norm

for root, dirs, files in os.walk(dirname):
    for filename in files:
        dir = root.split("/")[-1]
        input_filepath = os.path.join(root,filename)
        print(input_filepath)
        img = tiff.imread(input_filepath)
        norm_img = min_max_normalize(img)
        make_dir(os.path.join(o_dir,dir))
        output_filepath = os.path.join(o_dir,dir,filename)
        tiff.imsave(output_filepath,norm_img)


In [24]:
import os
import tifffile as tiff
import cv2
dirname = '/nas/k8s/dev/research/intern/jhmin/test_project/Datasets/SAR/Capella_uint16_Norm/Capella_0.5m_resize/'
large_size = 2000
for root, dirs, files in os.walk(dirname):
    for filename in files:
        dir = root.split("/")[-1]
        input_filepath = os.path.join(root,filename)
        
        img = cv2.imread(input_filepath)
        # print(img.shape, input_filepath)
        size, w ,c  = img.shape

        if size < large_size:
            large_size = size
        # print(large_size, input_filepath)

print(large_size)

606


In [2]:
import os
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

def calculate_psnr(img1, img2):
    """이미지 간 PSNR 계산"""
    return peak_signal_noise_ratio(img1, img2)

def compare_psnr_in_folders(folder1, folder2):

    psnr_values = []

    # 폴더 내의 이미지 파일 목록 가져오기
    images1 = os.listdir(folder1)
    images2 = os.listdir(folder2)

    # 이미지 파일들 간의 PSNR 계산
    for img_name1, img_name2 in zip(images1, images2):
        img_path1 = os.path.join(folder1, img_name1)
        img_path2 = os.path.join(folder2, img_name2)

        # 이미지 읽기
        img1 = cv2.imread(img_path1, cv2.IMREAD_COLOR)
        h,w,c  = img1.shape
        img2 = cv2.imread(img_path2, cv2.IMREAD_COLOR)

        img2 = cv2.resize(img2, dsize=(h,w), interpolation=cv2.INTER_CUBIC)

        # PSNR 계산
        psnr_value = calculate_psnr(img1, img2)
        psnr_values.append(psnr_value)

    # 평균 PSNR 계산
    average_psnr = np.mean(psnr_values)

    return psnr_values, average_psnr

# 두 폴더에 대한 PSNR 비교
folder1_path = "/nas/k8s/dev/research/intern/jhmin/test_project/Datasets/SAR/Capella_uint16_Norm/Capella_0.5m_resize_test/"  # 폴더1 경로로 수정
folder2_path = "/nas/k8s/dev/research/intern/jhmin/test_project/Datasets/SAR/Capella_uint16_Norm/Capella_1m_test/"  # 폴더2 경로로 수정

psnr_values, average_psnr = compare_psnr_in_folders(folder1_path, folder2_path)

print("PSNR values for each image:", psnr_values)
print("Average PSNR:", average_psnr)

PSNR values for each image: [9.319925298299902, 9.323173178583872, 9.783847179895995, 10.166962440981678, 10.012589235098176, 12.112449973935064, 8.513173492997385, 12.479843486396142, 10.14720818716691, 11.278899868483535, 11.602040761022408, 9.993059049244618, 8.073365072577252, 10.040855438730997, 9.472114635677274, 10.673141147079859, 9.689687734226723, 9.795273704075228, 11.702085358077913, 10.575506306179998, 12.032714249877078, 11.655096509141881, 9.635203288290583, 10.573478280256163, 9.672966722236575, 10.73560111832647, 10.52708663517478, 10.086421081037454, 7.577844953472551, 11.63895945173898, 9.903175768267298, 9.642284057414178, 11.6722070038381, 9.527114294262152, 10.518443222603695, 11.779416845955337, 10.840763908342089, 10.474005196500894, 10.929500454354091, 10.905317878658394, 10.852487917523717, 10.756469380721418, 10.801521167783974, 10.068727468133556, 9.912308803309337, 11.198052091376635, 10.34534474448025, 9.67898837258609, 7.432280194990693, 10.09911216359056

In [2]:
import splitfolders
input = "/nas/k8s/dev/research/intern/jhmin/test_project/Datasets/paired_eo/"
output = ""
splitfolders.ratio(input,input, seed=43, ratio=(0.8,0.1,0.1))

Copying files: 600 files [00:07, 83.59 files/s] 


In [46]:
import pickle
fname = "/nas/k8s/dev/research/intern/jhmin/test_project/despeckle/SRResNet/weights_pipal/settings1.pkl"
with open(fname, 'rb') as f:
    loaded_data = pickle.load(f)

# 읽어온 데이터 사용
print(loaded_data)

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.7608e-05,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 8.3692e-08, 2.4234e-04, 8.1181e-06,
        2.0337e-05, 5.9095e-04, 1.5344e-06, 5.4888e-03, 6.2769e-05, 3.0687e-07,
        9.2577e-04, 1.0043e-06, 2.7897e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        5.5795e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        6.2354e-03, 3.7103e-06, 8.9271e-07, 9.7641e-07, 1.5901e-06, 3.8189e-01,
        4.2659e-03, 5.6674e-02, 1.0992e-03, 3.4612e-04, 2.1585e-02, 4.7902e-04,
        1.1891e-02, 5.6185e-05, 7.1983e-04, 5.7468e-06, 0.0000e+00, 4.1846e-07,
        0.0000e+00, 0.0000e+00, 1.6459e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 6.2526e-04, 1.2710e-04, 8.3552e-05, 6.9743e-07, 1.8412e-06,
        2.6980e-02, 4.0211e-01, 6.1965e-03, 4.8953e-02, 1.5333e-02, 1.9891e-05,
        6.0453e-05, 1.9921e-03, 3.4735e-04, 4.5027e-03, 2.1481e-06, 2.7060e-06,
        2.6781e-06, 1.6180e-06, 4.1846e-

In [47]:
import torch
torch.topk(loaded_data, 5)

torch.return_types.topk(
values=tensor([0.4021, 0.3819, 0.0567, 0.0490, 0.0270]),
indices=tensor([61, 35, 37, 63, 60]))

In [ ]:
import numpy as np
np.float64(1/75)

In [ ]:
import random
def select_indices(data):
    indices = []
    for dimension in data:
        choice = random.choices(range(len(dimension)), dimension)
        indices.append(choice[0])
    return indices

# 무작위로 선택된 인덱스를 가져옵니다.
selected_indices = select_indices(loaded_data)

In [ ]:
selected_indices

In [ ]:
topk = ([0.0927, 0.0898, 0.0831, 0.0569, 0.0471],
([ 5,  8, 29,  7, 32]))

# PSNR 뽑기

In [14]:
import json

# JSON 파일에서 데이터 읽어오기
with open('/nas/k8s/dev/research/intern/jhmin/test_project/despeckle/ESRGAN-PyTorch/Test_output/RRDBNet_x4-CTD.json', 'r') as file:
    a_data = json.load(file)

with open('/nas/k8s/dev/research/intern/jhmin/test_project/despeckle/ESRGAN-PyTorch/Test_output/RRDBNet_x4-uniform.json', 'r') as file:
    b_data = json.load(file)

psnr_diff_data = []
for a_info in a_data:
    for b_info in b_data:
        if a_info['filename'] == b_info['filename']:
            psnr_diff = abs(a_info['psnr'] - b_info['psnr'])
            psnr_diff_data.append({'filename': a_info['filename'], 'psnr_diff': psnr_diff})

# PSNR 차이 기준으로 정렬
sorted_by_psnr_diff = sorted(psnr_diff_data, key=lambda x: x['psnr_diff'], reverse=True)

# 상위 5개 이미지 출력
print("Top 5 images with the largest PSNR difference:")
for i, info in enumerate(sorted_by_psnr_diff[:5], start=1):
    print(f'Top {i}: {info["filename"]} - PSNR Difference: {info["psnr_diff"]:.3f}dB')


Top 5 images with the largest PSNR difference:
Top 1: 0280.png - PSNR Difference: 0.382dB
Top 2: 0019.png - PSNR Difference: 0.274dB
Top 3: 0078.png - PSNR Difference: 0.245dB
Top 4: 0063.png - PSNR Difference: 0.228dB
Top 5: 0255.png - PSNR Difference: 0.208dB
